In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 5.0MB/s 


In [ ]:
from bs4 import BeautifulSoup
import lxml
import requests
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import re

In [ ]:
!apt-get update 
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
def scrape_bawangmerah_pipeline(url,g_colabs=True):
    '''
    documentation !
    this code tested on python 3.7
    platform : Google Chrome
    IDE : Google Colabs
    
    WARNING !!
    must use google chrome to run
    works best with google colabs

    RETURNS:
    order = df_kemarin,df_sekarang,source_code
    df_kemarin : dataframe berisi data untuk tanggal kemarin
    df_sekarang : dataframe berisi data untuk tanggal sekarang
    source_code : scraped site's source code

    Further Development:
    - comprehensive spice data scraping 
    - visualization features
    - list of spice available
    '''
    #instancing chrome webdriver and the argument for g_colabs
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    if g_colabs == True:
        driver = webdriver.Chrome('chromedriver',options=chrome_options)
    else:
        driver = webdriver.Chrome()
    

    link_init = url
    #request the sourcecode
    driver.get(link_init)
    #instancing the BS scraping using lxml parser
    source = BeautifulSoup(driver.page_source,features='lxml')

    #proper tags that cover data needed
    base_div = source.find('div',class_ = 'carousel slide')
    #element containing nama pasar
    all_name = base_div.find_all('div',class_='pc-city')
    pasar = []
    for name in all_name:
        pasar.append(name.text)
    #element containing raw price data and date
    all_price = base_div.find_all('div',class_ = 'pc-price')
    price_tag = []
    for price in all_price:
        price_tag.append(price.text)
    #extracting price with regex
    harga_atas = []
    harga_bawah = []
    pattern1 = re.compile(r'(\d+\.\d{3}).+(\d\d\.\d+)')
    for i in price_tag:
        matches = pattern1.finditer(i)
        for j in matches:
            harga_atas.append(j.group(1))
            harga_bawah.append(j.group(2))
    #extracting date with regex
    tanggal_atas = []
    tanggal_bawah = []
    pattern2 = re.compile(r'(\d{2}\s[a-zA-Z]+\s\d{4}).+(\d{2}\s[a-zA-Z]+\s\d{4})')
    for i in price_tag:
        matches = pattern2.finditer(i)
        for j in matches:
            tanggal_atas.append(j.group(1))
            tanggal_bawah.append(j.group(2))

    komoditas = ['Bawang Merah' for i in range(len(pasar))]
    df_kemarin = pd.DataFrame(data={'Komoditas':komoditas,'Tanggal':tanggal_bawah,'Nama_Pasar':pasar,'Harga':harga_bawah})
    df_sekarang = pd.DataFrame(data={'Komoditas':komoditas,'Tanggal':tanggal_atas,'Nama_Pasar':pasar,'Harga':harga_atas})
    return df_kemarin,df_sekarang,source

In [ ]:
df = scrape_bawangmerah_pipeline('https://infopangan.jakarta.go.id/publik/dashboard/12')
df.head()

,Komoditas,Nama_Pasar,Harga
0,Bawang Merah,Pasar Induk Kramat Jati,Harga 22 Mei 2021: Rp22.000/kgHarga 21 Mei 202...
1,Bawang Merah,Pasar Senen Blok III - VI,Harga 22 Mei 2021: Rp35.000/kgHarga 21 Mei 202...
2,Bawang Merah,Pasar Jembatan Merah,Harga 22 Mei 2021: Rp40.000/kgHarga 21 Mei 202...
3,Bawang Merah,Pasar Sunter Podomoro,Harga 22 Mei 2021: Rp44.000/kgHarga 21 Mei 202...
4,Bawang Merah,Pasar Rawa Badak,Harga 22 Mei 2021: Rp40.000/kgHarga 21 Mei 202...


# testing and pipeline engineering

In [ ]:
#navigate to dropdown element
dropdown_barang = driver.find_element_by_class_name('form-control')
dropdown_barang.click()
#access dropdown simulation
dd_barang_list = dropdown_barang.parent.find_elements_by_tag_name('option')
#perform click simulation
dd_barang_list[12].click()


ele_beras = WebDriverWait(driver,5).until(EC.presence_of_all_elements_located((By.CLASS_NAME,"pc-price")))
ele_pasar = WebDriverWait(driver,5).until(EC.presence_of_all_elements_located((By.CLASS_NAME,"pc-city")))


In [ ]:
def get_barang(input_barang):
    daftar_barang = {}
    for i,j in enumerate(input_barang):
        daftar_barang[i] = j
    kamus_barang = [k for k in daftar_barang.items()]
    return kamus_barang

get_barang(barang)

[(0, 'Beras IR. I (IR 64)'),
 (1, 'Beras IR. II (IR 64) Ramos'),
 (2, 'Beras IR. III (IR 64)'),
 (3, 'Beras Muncul .I'),
 (4, 'Beras IR 42/Pera'),
 (5, 'Beras Setra I/Premium'),
 (6, 'Minyak Goreng (Kuning/Curah)'),
 (7, 'Cabe Merah Keriting'),
 (8, 'Cabe Merah Besar (TW)'),
 (9, 'Cabe Rawit Merah'),
 (10, 'Cabe Rawit Hijau'),
 (11, 'Bawang Merah'),
 (12, 'Bawang Putih'),
 (13, 'Ayam Broiler/Ras'),
 (14, 'Telur Ayam Ras'),
 (15, 'Gula Pasir'),
 (16, 'Tepung Terigu'),
 (17, 'Garam Dapur'),
 (18, 'Kentang (sedang)'),
 (19, 'Tomat Buah'),
 (20, 'Kelapa Kupas'),
 (21, 'Semangka'),
 (22, 'Jeruk Medan'),
 (23, 'Daging Sapi Has (Paha Belakang)'),
 (24, 'Daging Sapi Murni (Semur)'),
 (25, 'Daging Kambing'),
 (26, 'Daging Babi Berlemak'),
 (27, 'Ikan Bandeng (sedang)'),
 (28, 'Ikan Mas'),
 (29, 'Ikan Lele'),
 (30, 'Gas Elpiji 3kg'),
 (31, 'Susu Bubuk Bendera 400gr'),
 (32, 'Susu Bubuk Dancow 400gr'),
 (33, 'Susu Kental Bendera 200gr'),
 (34, 'Susu Kental Enak 200gr'),
 (35, 'Margarin Blueband C

# pipeline result

In [ ]:
df = scrape_bawangmerah_pipeline('https://infopangan.jakarta.go.id/publik/dashboard/12')
df[1]

,Komoditas,Tanggal,Nama_Pasar,Harga
0,Bawang Merah,22 Mei 2021,Pasar Induk Kramat Jati,22.000
1,Bawang Merah,22 Mei 2021,Pasar Senen Blok III - VI,35.000
2,Bawang Merah,22 Mei 2021,Pasar Jembatan Merah,40.000
3,Bawang Merah,22 Mei 2021,Pasar Sunter Podomoro,44.000
4,Bawang Merah,22 Mei 2021,Pasar Rawa Badak,40.000
5,Bawang Merah,22 Mei 2021,Pasar Grogol,36.000
6,Bawang Merah,22 Mei 2021,Pasar Glodok,45.000
7,Bawang Merah,22 Mei 2021,Pasar Minggu,40.000
8,Bawang Merah,22 Mei 2021,Pasar Mayestik,45.000
9,Bawang Merah,22 Mei 2021,Pasar Pramuka,40.000


'ssss'